<a href="https://colab.research.google.com/github/albertomariapepe/CGA-PointNet/blob/main/CGA_PoseNet_(Cambridge_Landmarks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Defining CGA-PoseNet

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers.merge import concatenate
import tensorflow as tf
import keras

#Loading GoogLeNet with ImageNet weights

from keras.applications.inception_v3 import InceptionV3
model = InceptionV3(classifier_activation = None, weights = "imagenet", input_tensor=Input(shape=(224, 224, 3)))
# summarize the model
model.summary()

In [ ]:
#modifying the last layer

x2 = tf.keras.layers.Dense(2048)(model.layers[-1].output)
output2 = tf.keras.layers.Dense(8)(x2)
CGAPoseNet = tf.keras.Model(inputs=model.input, outputs=output2)
CGAPoseNet.summary()

# Converting Camera Poses into Motors


In [ ]:
FOLDER = "KingsCollege" #Change the name to change the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install clifford

In [ ]:
from clifford.g3c import *
from math import sqrt
import numpy as np

lambda_coeff = 200

#functions to convert quaternions coefficients into to GA rotors
def q2S(*args):
    '''
    convert tuple of quaternion coefficients to a spinor'''
    q = args
    return q[0] + q[1]*e13 +q[2]*e23 + q[3]*e12

#From Euclidean to 1D Up CGA. 
#function implementing Eq. 6 (convert a vector in Euclidean space into a rotor 
#in spherical space)
def translation_rotor(a, L = lambda_coeff):
    Ta = (L + a*e4)/(sqrt(L**2 + a**2))
    return Ta

#From Euclidean to 1D Up CGA. function implementing the Eq. 10 (X = f(x))
def up1D(x, L = lambda_coeff):
    Y = (2*L / (L**2 + x**2))*x + ((L**2-x**2)/(L**2 + x**2))*e4
    return Y

#From 1D Up CGA to Euclidean. function implementing the inverse of Eq. 10 (x = f^{-1}(X))
def down1D(Y, x, L = lambda_coeff):
    alpha = (2*L/(L**2 + x**2))
    beta  =  (L**2 - x**2)/(L**2 + x**2)
    x = (Y - beta*e4)/alpha

    return x

In [ ]:
#reads the dataset labels and converts them into motors (Train set)

list_of_lines = open("/content/drive/MyDrive/"+ FOLDER + "/dataset_train.txt").readlines()
newfile = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_train.txt", "w")

position_train = []

for i in range(0,3):
    newfile.write(list_of_lines[i])
    newfile.write("\n")

for i in range(3, len(list_of_lines)):

    a = []
    a = list_of_lines[i].split()

    position_train = np.append(position_train, [float(a[1]), float(a[2]), float(a[3])])

    Ta = translation_rotor(float(a[1])*e1 + float(a[2])*e2 + float(a[3])*e3)
    R = q2S(float(a[4]), float(a[5]), float(a[6]), float(a[7]))

    M = Ta*R
    
    a[1] = M[0] 
    a[2] = M[6]
    a[3] = M[7]
    a[4] = M[8]
    a[5] = M[10]
    a[6] = M[11]
    a[7] = M[13]
    a.append(M[26])

    b = " ".join(map(str, a))
    list_of_lines[i] = b
    newfile.write(list_of_lines[i])
    newfile.write("\n")

    if i == 3:
        print(a)
        print(list_of_lines[i])

print(i)
position_train = np.reshape(position_train, (-1, 3))

## Loading Train-Test Data

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
 
x_train = []

#reads the dataset frames, reshapes them and normalizes them  (Train Set)

list_train = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_train.txt").readlines() # no need for closing, python will do it for you
print(len(list_train))
for i in range(6,len(list_train)):
    if i % 100 == 0:
        print(i)
    a = list_train[i].split()
    img = cv2.imread("/content/drive/MyDrive/"+ FOLDER + "/" + a[0])
    
    resized = cv2.resize(img, (224, 224))
    normalized = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    x_train = np.append(x_train, normalized)


# Output img with window name as 'image'
#cv2_imshow(img)
cv2_imshow(resized)

np.save("x_train.npy", x_train)

In [ ]:
#reads the dataset labels and converts them into motors (Test Set)

list_of_lines = open("/content/drive/MyDrive/"+ FOLDER + "/dataset_test.txt").readlines() # no need for closing, python will do it for you
newfile = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_test.txt", "w")

position_test = []
for i in range(0,3):
    newfile.write(list_of_lines[i])
    newfile.write("\n")

for i in range(3, len(list_of_lines)):

    a = []
    a = list_of_lines[i].split()

    #print(a)

    position_test = np.append(position_test, [float(a[1]), float(a[2]), float(a[3])])


    Ta = translation_rotor(float(a[1])*e1 + float(a[2])*e2 + float(a[3])*e3)
    R = q2S(float(a[4]), float(a[5]), float(a[6]), float(a[7]))

    M = Ta*R
    
    a[1] = M[0] 
    a[2] = M[6]
    a[3] = M[7]
    a[4] = M[8]
    a[5] = M[10]
    a[6] = M[11]
    a[7] = M[13]
    a.append(M[26])

    #print(a)

    b = " ".join(map(str, a))
    list_of_lines[i] = b
    newfile.write(list_of_lines[i])
    newfile.write("\n")

    if i == 3:
        print(a)
        print(list_of_lines[i])

position_test = np.reshape(position_test, (-1, 3))

In [ ]:
x_test = []

#reads the dataset frames, reshapes them and normalizes them  (Train Set)

list_test = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_test.txt").readlines() 
print(len(list_test))
for i in range(6, len(list_test)):
    if i % 100 == 0:
        print(i)
    a = list_test[i].split()

    img = cv2.imread("/content/drive/MyDrive/"+ FOLDER + "/" + a[0])
    resized = cv2.resize(img, (224, 224))
    normalized = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    x_test = np.append(x_test, normalized)
 

cv2_imshow(resized)

np.save("x_test.npy", x_test)

In [ ]:
y_train = []

#loads train and test labels in motor form

list_of_lines = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_train.txt").readlines()
for i in range(6, len(list_of_lines)):
    if i % 100 == 0:
        print(i)
    a = list_of_lines[i].split()

    y_train = np.append(y_train, float(a[1]))
    y_train = np.append(y_train, float(a[2]))
    y_train = np.append(y_train, float(a[3]))
    y_train = np.append(y_train, float(a[4]))
    y_train = np.append(y_train, float(a[5]))
    y_train = np.append(y_train, float(a[6]))
    y_train = np.append(y_train, float(a[7]))
    y_train = np.append(y_train, float(a[8]))


print("****")
y_test = []
list_of_lines = open("/content/drive/MyDrive/"+ FOLDER + "/new_dataset_test.txt").readlines()
for i in range(6, len(list_of_lines)):
    if i % 100 == 0:
        print(i)
    a = list_of_lines[i].split()
    y_test = np.append(y_test, float(a[1]))
    y_test = np.append(y_test, float(a[2]))
    y_test = np.append(y_test, float(a[3]))
    y_test = np.append(y_test, float(a[4]))
    y_test = np.append(y_test, float(a[5]))
    y_test = np.append(y_test, float(a[6]))
    y_test = np.append(y_test, float(a[7]))
    y_test = np.append(y_test, float(a[8]))


np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)

In [ ]:
#224 x 224 x 3 is the size of each frame
x_train = np.reshape(x_train, (-1, 224, 224, 3)) 
x_test = np.reshape(x_test, (-1, 224, 224, 3))

#8 is the number of motor coefficients
y_train = np.reshape(y_train, (-1, 8)) 
y_test = np.reshape(y_test, (-1, 8))

In [ ]:
#plotting the camera pose in Euclidean (original ground truth) and spherical space
#If Lambda is chose large enough, the two traces should match

import matplotlib.pyplot as plt

origin = e4

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

trans_GA = []
trans_GT = []

position = position_train

positional_error = []

T_or = []
T_pred = []
for i in range(len(y_train)):
    x = 1

    X = y_train[i]

    M_real = X[0] + X[1]*e12 + X[2]*e13 + X[3]*e14 + X[4]*e23 + X[5]*e24 + X[6]*e34 + X[7]*e1234

    T = M_real * origin * ~M_real
    t = down1D(T, x)

    mae = np.mean(np.abs(np.array([t[1], t[2], t[3]]) - np.array([position[i,0], position[i,1], position[i,2]])))

    positional_error  = np.append(positional_error , mae)

    trans_GA = np.append(trans_GA, np.array([t[1], t[2], t[3]]))
    trans_GT = np.append(trans_GT, np.array([position[i,0], position[i,1], position[i,2]]))

    ax.scatter(t[1], t[2], t[3], s = 20, c = "r")
    ax.scatter(position[i,0], position[i,1], position[i,2], s = 20, c = "b", alpha = 0.5)

    T_or = np.append(T_or, [t[1], t[2], t[3]])
    T_pred = np.append(T_pred, [position[i,0], position[i,1], position[i,2]])


ax.set_xlabel("X", fontsize = 40.0)
ax.set_ylabel("Y", fontsize = 40.0)
ax.set_zlabel("Z", fontsize = 40.0)

plt.show()       

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# Training

In [ ]:
#defining hyperparameters

nb_epoch = 100
batch_size = 64 

initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.90,
    staircase=True)

#compiling the model
CGAPoseNet.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                 loss="mse", run_eagerly=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

#training
model_train = CGAPoseNet.fit(x = x_train, y =y_train,
                           validation_split = 0.2,
                            epochs=nb_epoch,
                            verbose=1,
                            shuffle=True,
                            callbacks = es_callback,
                            batch_size=batch_size)

#plotting losses
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(0,np.size(loss))

plt.figure()
plt.plot(epochs, loss, 'b-', label='Training loss')
plt.plot(epochs, val_loss, 'r-', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

In [ ]:
#storing losses
np.save("train_loss.npy", loss)
np.save("val_loss.npy", val_loss)

In [ ]:
#saving weights
CGAPoseNet.save('weights.h5')
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Testing

In [ ]:
#model = keras.models.load_model('weights.h5')

In [ ]:
#prediction step
y_pred = CGAPoseNet.predict(x_test)
np.save("y_pred.npy", y_pred)

In [ ]:
#Storing the MSE between \hat{M}, M over the test set

MSE = []

tot = 0
cnt = 0
for i in range(len(y_test)):
    mse = (np.square(y_test[i] - y_pred[i])).mean()
    
    MSE = np.append(MSE, mse)

    #printing the first 20 motors M, \hat{M} if the MSE between them is close 
    if cnt < 20 and mse < 0.0008:
        print("original:" , y_test)

        X = y_test[i]
        Y = y_pred[i]

        M_real = X[0] + X[1]*e12 + X[2]*e13 + X[3]*e14 + X[4]*e23 + X[5]*e24 + X[6]*e34 + X[7]*e1234
        M_pred = Y[0] + Y[1]*e12 + Y[2]*e13 + Y[3]*e14 + Y[4]*e23 + Y[5]*e24 + Y[6]*e34 + Y[7]*e1234

        print("prediction:", y_pred[i])
        print("****")
        cnt += 1
    
    tot += mse


print(tot)
np.save("MSE.npy", MSE)

In [ ]:
#evaluating positional and rotational error (see Section 4.4 Metrics)

origin = e4
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
list_of_lines = open("/content/drive/MyDrive/"+ FOLDER + "/dataset_test.txt").readlines() # no need for closing, python will do it for you

positional_error = []
rotational_error = []

translation = []
translation_pred = []

rotation = []
rotation_pred = []
for i in range(len(y_test)):
    a = []
    a = list_of_lines[i+3].split()

    #x is required by the function down1D
    x = float(a[1])*e1 + float(a[2])*e2 + float(a[3])*e3

    X = y_test[i]
    Y = y_pred[i]

    #construct M and \hat{M}
    M_real = X[0] + X[1]*e12 + X[2]*e13 + X[3]*e14 + X[4]*e23 + X[5]*e24 + X[6]*e34 + X[7]*e1234
    M_pred = Y[0] + Y[1]*e12 + Y[2]*e13 + Y[3]*e14 + Y[4]*e23 + Y[5]*e24 + Y[6]*e34 + Y[7]*e1234

    #normalizing
    M_pred = M_pred / sqrt((M_pred* ~M_pred)[0])

    #predicted and real displacement vector \hat{D}, D in spherical space
    S = M_pred * origin * ~M_pred
    T = M_real * origin * ~M_real

    #predicted and real displacement vector \hat{d}, d in Euclidean space
    s = down1D(S, x)
    t = down1D(T, x)

    #POSITIONAL ERROR
    mae = np.mean(np.abs(np.array([t[1], t[2], t[3]]) - np.array([s[1], s[2], s[3]])))

    positional_error  = np.append(positional_error , mae)

    translation = np.append(translation, np.array([t[1], t[2], t[3]]))
    translation_pred = np.append(translation_pred, np.array([s[1], s[2], s[3]]))


    #plotting the camera trace
    ax.scatter(t[1], t[2], t[3], s = 20, c = "r")
    ax.scatter(s[1], s[2], s[3], s = 20, c = "b", alpha = 0.5)

    Tup = translation_rotor(t[1]*e1 + t[2]*e2 + t[3]*e3)
    Sup = translation_rotor(s[1]*e1 + s[2]*e2 + s[3]*e3)

    #predicted and real rotors \hat{R}, R
    R_pred = ~Sup* M_pred 
    R_real = ~Tup* M_real

    if (R_real * ~R_pred)[0] > 1:
        error = (np.arccos(1))*360/(2*np.pi)
    elif (R_real * ~R_pred)[0] < -1:
        error = (np.arccos(-1))*360/(2*np.pi)
    else:
        #ROTATIONAL ERROR
        error = (np.arccos((R_real * ~R_pred)[0]))*360/(2*np.pi)

    rotational_error = np.append(rotational_error, error)

    rotation = np.append(rotation, np.array([R_real[0], R_real[6], R_real[7], R_real[10]]))
    rotation_pred = np.append(rotation_pred, np.array([R_pred[0], R_pred[6], R_pred[7], R_pred[10]]))
 

plt.show()


#storing rotational and translational errors
np.save("translation_error.npy", positional_error)
np.save("rotational_error.npy", rotational_error)

#storing original and predicted translations
np.save("T.npy", translation)
np.save("S.npy", translation_pred) 

#storing original and predicted rotations
np.save("R.npy", rotation)
np.save("Q.npy", rotation_pred)          

In [ ]:
#plotting the camera orientation (coefficients e_{12}, e_{13}, e_{23} of rotor R)
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

N=200
stride=1

u = np.linspace(0, 2 * np.pi, N)
v = np.linspace(0, np.pi, N)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x, y, z, linewidth=0.0, alpha = 0.1, cstride=stride, rstride=stride)

ax.scatter(0, 0, 0, c = "k", marker = "s", label = "O")

rotation = np.reshape(rotation, (-1, 4))
rotation_pred = np.reshape(rotation_pred, (-1, 4))
ax.scatter(rotation[:,1], rotation[:, 2], rotation[:,3], s = 15, c = "r")
ax.scatter(rotation_pred[:,1], rotation_pred[:, 2], rotation_pred[:,3], s = 15, c = "b")
plt.show()

In [ ]:
print(np.median(positional_error))
print(np.mean(  positional_error))

In [ ]:
print(np.median(rotational_error))
print(np.mean(  rotational_error))